In [29]:
import os
import re
import pickle
import numpy as np
import json
def get_Models_get_predictions(num, temp, hour, description, day):
    DayDict={'Friday': 0,
     'Wednesday': 6,
     'Saturday': 2,
     'Tuesday': 5,
     'Thursday': 4,
     'Monday': 1,
     'Sunday': 3}

    description_dict= {'light rain': 5,
     'broken clouds': 0,
     'clear sky': 1,
     'mist': 7,
     'few clouds': 2,
     'moderate rain': 8,
     'scattered clouds': 10,
     'overcast clouds': 9,
     'light intensity drizzle': 4,
     'light snow': 6,
     'heavy intensity rain': 3}
    #encode categorical data to numeric for model 
    description_encoded = description_dict.get(description)
    day_encoded = DayDict.get(day)
    
    Bikesfiles={}
    Standsfiles={}
    
    
    if hour <11:
        hours = list(range(hour, hour+12, 1))
       
    else:
        hours_left = 23- hour
        hours_start = list(range(hour, hour+hours_left+1, 1))
        hours_to_add = 12 - hours_left
        other_hours = list(range(hours_to_add))
        hours = hours_start + other_hours
        

    for file in os.listdir():
        if file.endswith("Bikes.pkl"):
            regex = re.compile(r'\d+')
            station = [int(x) for x in regex.findall(file)]
            Bikesfiles[station[0]] = file

        elif file.endswith("Stands.pkl"):
            regex = re.compile(r'\d+')
            station = [int(x) for x in regex.findall(file)]
            Standsfiles[station[0]] = file

    BikesModelName = Bikesfiles.get(num)
    StandsModelName = Standsfiles.get(num)

    with open(BikesModelName, 'rb') as handle:
        modelBikes = pickle.load(handle)

    with open(StandsModelName, 'rb') as handle:
        modelStands = pickle.load(handle)
    predictionsBikes={}
    for x in hours:
        array = np.array([temp, x, description_encoded, day_encoded])
        x_test = array.reshape(1, -1)
        p = modelBikes.predict(x_test)
        predictionsBikes[x] =p.tolist()
    
    predictionsStands={}
    for x in hours:
        array = np.array([temp, x, description_encoded, day_encoded])
        x_test = array.reshape(1, -1)
        p = modelStands.predict(x_test)
        predictionsStands[x] =p.tolist()
    
  
    return json.dumps(predictionsBikes), json.dumps(predictionsStands)


In [30]:
get_Models_get_predictions(7, 3, 18, 'light snow', 'Thursday')

('{"18": [10], "19": [10], "20": [3], "21": [2], "22": [2], "23": [2], "0": [8], "1": [8], "2": [8], "3": [8], "4": [15], "5": [9], "6": [10]}',
 '{"18": [19], "19": [19], "20": [26], "21": [27], "22": [27], "23": [27], "0": [21], "1": [21], "2": [21], "3": [14], "4": [14], "5": [19], "6": [19]}')